# Select database (i.e. staging, production, local authenticated, local unauthenticated)

In [1]:
import requests
from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.common import ProjectContext
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    LOCAL_AUTHENTICATED = "local_authenticated"

obi_one_api_url = "http://127.0.0.1:8100"
virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
project_id="2720f785-a3a2-4472-969d-19a53891c817"

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED
# database_environment = DatabaseEnvironment.LOCAL_AUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "https://127.0.1:8000/api/entitycore"

elif DatabaseEnvironment.LOCAL_AUTHENTICATED:
    """
    Not yet tested.
    - Change APP_DISABLE_AUTH to False in entitycore/.env.run-local
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk morphology.ipynb example)
    """
    entitycore_api_url = "https://127.0.1:8000/api/entitycore"
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
print(token)
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJZVTEyTEoyNG1sRFRieURlWXgyaGU5RkQzbldkWlBSV2piSVVpa2hocVFVIn0.eyJleHAiOjE3NDkxMjkxOTAsImlhdCI6MTc0OTEyNTU5MCwianRpIjoib25ydGFjOmMzMDY1Yzk3LTIyYjYtNGNlZi04MDI5LTExZjRkMTAxZDRkZiIsImlzcyI6Imh0dHBzOi8vc3RhZ2luZy5vcGVuYnJhaW5pbnN0aXR1dGUub3JnL2F1dGgvcmVhbG1zL1NCTyIsImF1ZCI6ImFjY291bnQiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJvYmktZW50aXR5c2RrLWF1dGgiLCJzaWQiOiIyOTBkMmM2ZC1jODYwLTQ3OWItYTE5Ny00NmI1ZWVkMmI1ZGQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdCoiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNibyJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUiLCJuYW1lIjoiSmFtZXMgSXNiaXN0ZXIiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJqYW1lcy1pc2Jpc3RlciIsImdpdmVuX25hbWUiOiJKYW1lcyIsImZhbWlseV9uYW1lIjoiSXNiaXN0ZXIifQ.YPPnHISem9LlVCbeNXtTYt1l0lmVFP-HzzAeM7c1TvIcVU9TH64anuxwSmSDrFpkbsfeL2CKRS94N

# Fetch a morphology

In [2]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {}
reconstruction_morphologies = client.search_entity(
    entity_type=ReconstructionMorphology, query=kwargs, token=token, limit=10
).all()

reconstruction_morphology_id = reconstruction_morphologies[0].id

# Call the declared morphology metrics endpoint

In [3]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/neuron-morphology-metrics/{reconstruction_morphology_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8100): Max retries exceeded with url: /declared/neuron-morphology-metrics/e380cafe-c36b-43e1-b040-5c7ee34b0e72 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10b03e090>: Failed to establish a new connection: [Errno 61] Connection refused'))

# Call the generated morphology metrics endpoint

In [ ]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/morphology-metrics-run-grid"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request body — adjust this with the actual morphology metrics form data!
# Placeholder example:
request_body = {
  "type": "MorphologyMetricsForm",
  "initialize": {
    "type": "MorphologyMetricsForm.Initialize",
    "morphology": {
      "id_str": str(reconstruction_morphology_id),
      "additionalProp1": {}
    }
  }
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")